In [14]:
import numpy as np
import pandas as pd
import re
from ast import literal_eval

In [15]:
def metric(data_true, chatgpt_pred):
    name_list = ["person", "organization", "location", "miscellaneous"]
    all_truth_entity,hit_entity,all_pred_entity = 0,0,0
    label_entity_all, pred_entity_all = [], []
    category_result = {}
    entity_notin_sentence = []
    for name in name_list:
        category_result[name] = {}
        category_result[name]["single_hit"] = 0
        category_result[name]["single_pred"] = 0
        category_result[name]["single_true"] = 0

    for i in range(len(chatgpt_pred)):
        sentence = chatgpt_pred.loc[i, "text"]
        for name in name_list:
            truth_entity = data_true.loc[i, name]
            pred_entity = chatgpt_pred.loc[i, name]
            truth_entity_list = literal_eval(truth_entity)
            pred_entity_list = literal_eval(pred_entity)
            new_truth_list = truth_entity_list
            single_hit, single_pred, single_true = 0, 0, 0
            if pred_entity_list == ['None']:
                new_pred_list = []
            else:
                new_pred_list = pred_entity_list
                
            for index in range(len(new_pred_list)):
                if str(new_pred_list[index]) not in sentence:
                    entity_notin_sentence.append(new_pred_list[index])
                else:
                    if new_pred_list[index] in new_truth_list:
                        hit_entity += 1
                        single_hit += 1
                    all_pred_entity += 1
                    single_pred += 1
                    
            for index in range(len(new_truth_list)):
                if new_truth_list[index] != []:
                    all_truth_entity += 1
                    single_true += 1
            label_entity_all.append(new_truth_list)
            pred_entity_all.append(new_pred_list)
            category_result[name]["single_hit"] += single_hit
            category_result[name]["single_pred"] += single_pred
            category_result[name]["single_true"] += single_true

    P = hit_entity/all_pred_entity
    R = hit_entity/all_truth_entity
    F1 = 2*P*R/(P+R)

    result = {}
    result["all_truth_entity"] = all_truth_entity
    result["all_pred_entity"] = all_pred_entity
    result["hit_entity"] = hit_entity
    result["P"] = P
    result["R"] = R
    result["F1"] = F1
    print("entity not in sentence:", len(entity_notin_sentence))
    for name in name_list:
        category_result[name]["P"] = category_result[name]["single_hit"]/category_result[name]["single_pred"]
        category_result[name]["R"] = category_result[name]["single_hit"]/category_result[name]["single_true"]
        category_result[name]["F1"] = 2*category_result[name]["P"]*category_result[name]["R"]/(category_result[name]["P"]+category_result[name]["R"])
    return result, pred_entity_all, label_entity_all, category_result



In [17]:
# multiMM_gpt3.5_twitter2017_50-1_shot-4
chatgpt_pred = pd.read_csv("multiMM_gpt3.5_twitter2015_50-1_shot-4_predict_test.csv", sep="\t")
data_true = pd.read_csv("./obtain_image_caption/twitter2015_process_caption_test.csv", sep="\t")
chatgpt_pred = chatgpt_pred.fillna("['None']")
result, pred_entity_all, label_entity_all, category_result = metric(data_true, chatgpt_pred)
for k in result:
    print(k + ":", result[k])
for k in category_result:
    print(k + ":", category_result[k])

entity not in sentence: 576
all_truth_entity: 5116
all_pred_entity: 6896
hit_entity: 3370
P: 0.4886890951276102
R: 0.6587177482408131
F1: 0.5611055611055611
person: {'single_hit': 1508, 'single_pred': 2075, 'single_true': 1825, 'P': 0.7267469879518073, 'R': 0.8263013698630137, 'F1': 0.7733333333333334}
organization: {'single_hit': 550, 'single_pred': 2015, 'single_true': 839, 'P': 0.2729528535980149, 'R': 0.6555423122765197, 'F1': 0.38542396636299936}
location: {'single_hit': 1126, 'single_pred': 1689, 'single_true': 1728, 'P': 0.6666666666666666, 'R': 0.6516203703703703, 'F1': 0.6590576529119111}
miscellaneous: {'single_hit': 186, 'single_pred': 1117, 'single_true': 724, 'P': 0.1665174574753805, 'R': 0.2569060773480663, 'F1': 0.2020640956002173}
